[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

> This notebook runs YouTubeDNN on MovieLens dataset.

# Download dataset and install packages

In [1]:
!git clone https://github.com/Wp-Zhang/HandyRec.git
! pip install faiss-cpu

Cloning into 'HandyRec'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 134 (delta 49), reused 93 (delta 19), pack-reused 0
Receiving objects: 100% (134/134), 214.29 KiB | 30.61 MiB/s, done.
Resolving deltas: 100% (49/49), done.
     |████████████████████████████████| 8.6 MB 28.7 MB/s 


In [2]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! unzip -o ml-1m.zip

--2022-03-11 20:18:21--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  28.2MB/s    in 0.2s    

2022-03-11 20:18:22 (28.2 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


# Import relative packages

In [3]:
import sys
sys.path.append('./HandyRec/')

In [4]:
from handyrec.examples.utils import DataProcessor, sampledsoftmaxloss
from handyrec.models.match import YouTubeMatchDNN
from handyrec.features import DenseFeature, SparseFeature, SparseSeqFeature
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import faiss
from sklearn.preprocessing import StandardScaler

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load and prepare data

In [6]:
EMBEDDING_DIM = 128
SEQ_LEN = 40
BATCH_SIZE = 2**12
NEPOCH = 100

In [7]:
dp = DataProcessor('./ml-1m/')

In [8]:
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
lbd_features = ['gender','age','occupation', 'zip']
data = dp.preprocess_data(lbd_features)
dp.gen_data_set(features, data, seq_max_len=SEQ_LEN, negnum=0)

Encode User Sparse Feats: 100%|██████████| 4/4 [00:00<00:00, 160.23it/s]
Encode Item Sparse Feats: 0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  7.54it/s]
0it [00:00, ?it/s]


In [9]:
user_features = ['user_id','gender','age','occupation', 'zip']
movie_features = ['movie_id']
train, train_label, test, test_label = dp.load_dataset(user_features, movie_features)

Load movie Features: 100%|██████████| 1/1 [00:00<00:00, 330.18it/s]


In [10]:
# add example_age^2 as showed in the original paper
train['example_age_2'] = train['example_age']**2
test['example_age_2'] = test['example_age']**2

# Train model and export embeddings

In [11]:
# Standarize dense feature
stds = StandardScaler()
stds2 = StandardScaler()
train['example_age'] = stds.fit_transform(train['example_age'].reshape(-1,1)).reshape(-1)
test['example_age'] = stds.transform(test['example_age'].reshape(-1,1)).reshape(-1)
train['example_age_2'] = stds2.fit_transform(train['example_age_2'].reshape(-1,1)).reshape(-1)
test['example_age_2'] = stds2.transform(test['example_age_2'].reshape(-1,1)).reshape(-1)

In [12]:
feature_dim = {}
for feat in user_features:
    feature_dim[feat] = data['user'][feat].max()+1
for feat in movie_features:
    feature_dim[feat] = data['item'][feat].max()+1

In [13]:
user_dense_feats = ['example_age','example_age_2']
user_sparse_feats = ['user_id','gender','age','occupation', 'zip']

In [14]:
user_features = [SparseFeature(x, feature_dim[x], EMBEDDING_DIM) for x in user_sparse_feats] +\
                [DenseFeature(x) for x in user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', feature_dim['movie_id'], EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
item_id = SparseFeature('movie_id', feature_dim['movie_id'], EMBEDDING_DIM)

In [15]:
tf.compat.v1.disable_eager_execution()

In [16]:
model = YouTubeMatchDNN(
    user_features, item_id, num_sampled=100, 
    user_dnn_hidden_units=(256,128,EMBEDDING_DIM), dnn_dropout=0.2
)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model.compile(optimizer="Adam", loss=sampledsoftmaxloss)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(train, train_label,
                    batch_size=BATCH_SIZE, 
                    epochs=NEPOCH,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stop])
model.save_weights('youtubednn.h5')

Train on 845828 samples, validate on 93981 samples
Epoch 1/100
845828/845828 [==============================] - 19s 22us/sample - loss: 3.6998 - val_loss: 3.2906
Epoch 2/100
845828/845828 [==============================] - 10s 12us/sample - loss: 3.1211 - val_loss: 3.0341
Epoch 3/100
845828/845828 [==============================] - 11s 13us/sample - loss: 2.8345 - val_loss: 2.7408
Epoch 4/100
845828/845828 [==============================] - 10s 12us/sample - loss: 2.6295 - val_loss: 2.5277
Epoch 5/100
845828/845828 [==============================] - 9s 11us/sample - loss: 2.4836 - val_loss: 2.4129
Epoch 6/100
845828/845828 [==============================] - 9s 11us/sample - loss: 2.3767 - val_loss: 2.2957
Epoch 7/100
845828/845828 [==============================] - 9s 11us/sample - loss: 2.3164 - val_loss: 2.2291
Epoch 8/100
845828/845828 [==============================] - 9s 11us/sample - loss: 2.2336 - val_loss: 2.2162
Epoch 9/100
845828/845828 [==============================] - 9s 1

In [18]:
model.load_weights('youtubednn.h5')

In [19]:
all_item_model_input = {"movie_id": data['item']['movie_id'].values}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(6040, 128)
(3883, 128)


# Use Faiss to predict

In [20]:
index = faiss.IndexFlatIP(EMBEDDING_DIM)
index.add(item_embs)

In [21]:
D, I = index.search(np.ascontiguousarray(user_embs), 10)
s = []
for i, uid in enumerate(test['user_id']):
    try:
        pred = data['item']['movie_id'].values[I[i]].tolist()
        s.append(pred)
    except:
        print(i)
s = np.array(s)

In [22]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean(
        [apk(a, p, k) for a, p in zip(actual, predicted)]
    )


def rk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = sum([1 for r in actual if r in predicted])/len(actual)

    return score

def recall_at_k(actual, predicted, k=12):
    return np.mean(
        [rk(a, p, k) for a, p in zip(actual, predicted)]
    )

In [23]:
mapk(test_label, s, k=10)
# 0.026905721118469464 map@10 without `example_age` and `example_age_2`
# 0.039653362503941976  map@10 with `example_age` and `example_age^2`

0.039653362503941976

In [24]:
recall_at_k(test_label, s, k=10)
# 0.07466887417218543 reacall@10 without `example_age` and `example_age_2`
# 0.0939403973509934  reacall@10 with `example_age` and `example_age^2`

0.0939403973509934